# A100 7번 GPU 사용

In [1]:
!git config --global credential.helper store
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from IPython.display import Image, display, HTML
import PyPDF2
from tqdm import tqdm
import re
import json
import os
import torch
import random
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
data_path = '/workspace/data/grit_all_1.csv'

In [4]:
def extract_keywords(filename):
    name_wo_ext = re.sub(r'\.\w+$', '', filename)
    name_wo_ext = re.sub(r'\(([^)]*)\)', lambda m: m.group(1).replace(',', ''), name_wo_ext)
    tokens = re.split(r'[_+()\s]+', name_wo_ext)
    tokens = [t for t in tokens if not re.search(r'\d', t) and t]
    return ', '.join(tokens)

In [5]:
df = pd.read_csv(data_path)
df.columns = ['path', 'filename']
df['keyword'] = df['filename'].apply(extract_keywords)
df['path'] = df['path'].str.replace('\\', '/', regex=False)
df = df.drop(['filename'], axis=1)

In [6]:
dataset = DatasetDict({'train': Dataset.from_pandas(df[['keyword', 'path']])})
dataset

DatasetDict({
    train: Dataset({
        features: ['keyword', 'path'],
        num_rows: 110955
    })
})

In [7]:
dataset['train'][0]

{'keyword': 'ITCRC, 기간연장협약서',
 'path': 'Z:/1_일반행정/1_센터_규정_및_소개/1_설립_및_규정/1_ITCRC_협약서/ITCRC_기간연장협약서_200811.pdf'}

In [8]:
dataset['train'][4]

{'keyword': '자산보관위탁기간연장약정서',
 'path': 'Z:/1_일반행정/1_센터_규정_및_소개/1_설립_및_규정/1_ITCRC_협약서/자산보관위탁기간연장약정서_20090922.jpg'}

In [9]:
dataset['train'][47]

{'keyword': '자립화계획서',
 'path': 'Z:/1_일반행정/1_센터_규정_및_소개/1_설립_및_규정/2_자립화_관련/GEMS자립화계획서/20100820_제출/발표자료/자립화계획서_ver01.pptx'}

In [10]:
BASE_MODEL = 'google/gemma-2b-it'

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
def finetuning_before(keyword):
    messages = [
        {
            'role': 'user', 
            'content': (
                '아래는 키워드입니다.\n\n'
                '해당 키워드에 맞는 문서 경로를 알려주세요.\n'
                f'keyword: {keyword}\n'
                'path:'
            )
        }
    ]
    pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=256)
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    params = {'do_sample': True, 'temperature': 0.3, 'top_k': 40, 'top_p': 0.9, 'add_special_tokens': True}
    outputs = pipe(prompt, **params)
    result = outputs[0]['generated_text'][len(prompt):].strip()
    print(result)

In [12]:
finetuning_before(dataset['train'][0]['keyword'])

ITCRC의 문서 경로는 다음과 같습니다.

- ITCRC 홈페이지: path/to/itcrc.do
- 기간연장협약서 관련 문서: path/to/itcrc/agreement.do


In [13]:
finetuning_before(dataset['train'][4]['keyword'])

키워드 자산보관위탁기간연장약정서는 다음과 같은 문서 경로에 존재합니다.

* **법률정보시스템:** 자산보관위탁기간연장약정서 제3조
* **국민은행 고유정보 공개규:** 자산보관위탁기간연장약정서 제3조
* **국민은행 웹사이트:** 자산보관위탁기간연장약정서 제3조
* **경찰서의 주소:** 경찰서의 주소를 포함한 지역 행사지도를 확인할 수 있습니다.


In [14]:
finetuning_before(dataset['train'][47]['keyword'])

자립화계획서의 문서 경로는 다음과 같습니다.

* **C:\Users\username\Documents\자립화계획서\**

이 경로는 사용자의 개인 문서 저장소의 "자립화계획서" 폴더에 위치합니다.


In [15]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float
)

In [16]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
def generate_prompt(keyword):
    prompt = (
        '아래는 키워드입니다.\n\n'
        '해당 키워드에 맞는 문서 경로를 알려주세요.\n'
        f'keyword: {keyword}\n'
        'path:'
    )
    return [prompt]

In [18]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        max_steps=1000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/110955 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [19]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Step,Training Loss
100,0.857300
200,0.332000
300,0.108900
400,0.045600
500,0.027900
600,0.020000
700,0.015900
800,0.012900
900,0.011500
1000,0.010200


TrainOutput(global_step=1000, training_loss=0.1442343417406082, metrics={'train_runtime': 674.1872, 'train_samples_per_second': 5.933, 'train_steps_per_second': 1.483, 'total_flos': 2.4443766177792e+16, 'train_loss': 0.1442343417406082, 'epoch': 36.04})

In [20]:
ADAPTER_MODEL = 'lora_adapter'
trainer.model.save_pretrained(ADAPTER_MODEL)

In [21]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)
model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-sum-ko')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
BASE_MODEL = 'google/gemma-2b-it'
FINETUNE_MODEL = './gemma-2b-it-sum-ko'

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={'':0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
def inference(keyword):
    pipe = pipeline(
        "text-generation",
        model=finetune_model,
        tokenizer=tokenizer,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.2,
    )
    prompt = (
        '아래는 키워드입니다.\n\n'
        '해당 키워드에 맞는 문서 경로를 알려주세요.\n'
        f'keyword: {keyword}\n'
        'path:'
    )    
    output = pipe(prompt)[0]['generated_text']
    print(f"Keyword: {keyword}")
    print(output)

In [28]:
inference(dataset['train'][0]['keyword'])

Keyword: ITCRC, 기간연장협약서
아래는 키워드입니다.

해당 키워드에 맞는 문서 경로를 알려주세요.
keyword: ITCRC, 기간연장협약서
path:


In [29]:
inference(dataset['train'][4]['keyword'])

Keyword: 자산보관위탁기간연장약정서
아래는 키워드입니다.

해당 키워드에 맞는 문서 경로를 알려주세요.
keyword: 자산보관위탁기간연장약정서
path:


In [30]:
inference(dataset['train'][47]['keyword'])

Keyword: 자립화계획서
아래는 키워드입니다.

해당 키워드에 맞는 문서 경로를 알려주세요.
keyword: 자립화계획서
path:


In [31]:
inference('연가사용현황')

Keyword: 연가사용현황
아래는 키워드입니다.

해당 키워드에 맞는 문서 경로를 알려주세요.
keyword: 연가사용현황
path:
